In [24]:
import sys
sys.path.append('..')

import os
import shutil
from sqlalchemy import create_engine, text
import geopandas as gpd
import pandas as pd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
import simplekml

from utilities_amigocloud import AmigocloudFunctions

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [25]:
# leer los dos shps de siembra y puntos de planificacion
#PATH_SIEMBRA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\SIEMBRAS_2.shp'
#PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S09_MIERCOLES.shp'
PATH_KML = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN'

#gdf_siembras = gpd.read_file(PATH_SIEMBRA)
#gdf_cat = gpd.read_file(PATH_CAT)

In [26]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion_no_procesado():
    engine = obtener_engine()
    try:
        query = """
            SELECT id_os, geom, cod_ca, nom_ca, obs, procesado, producto, dosis, semana_plan
            FROM drones_control_bio.planificacion_ctrl_bio
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def get_catastro():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.catastro_drones
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_siembras():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.siembra
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener capa de siembras: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_por_id_os(id_os):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where os = {id_os}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_sin_planificar():
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where estado IS NULL
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_planificado():
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where estado = 'PLANIFICADO'
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def filtrar_os_y_quitar_caracteres(gdf, ors):
    # filtrar la semana de interes
    gdf_puntos_select = gdf[gdf['id_os'] == ors]
    gdf_puntos_sem = gdf_puntos_select.copy()
    #quita caracteres especiales
    gdf_puntos_sem['nom_ca'] = gdf_puntos_sem['nom_ca'].str.replace(r'[\r\n\t]', '', regex=True)
    return gdf_puntos_sem

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.planificacion_ctrl_bio
                SET procesado = true
                WHERE id_os = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id_os {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def marcar_como_planificado(id_lote):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.parte_diario_ctrl_bio
                SET estado = 'PLANIFICADO'
                WHERE id = :id_lote
            """)
            conn.execute(query, {"id_lote": id_lote})
            print(f"✔️ id_lote {id_lote} marcado como PLANIFICADO.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None


def crear_puntos_individuales(gdf_fila):
    # Obtener la geometría del único registro
    geom = gdf_fila.iloc[0][gdf_fila.geometry.name]
    # Verificar si es MultiPoint
    if isinstance(geom, MultiPoint):
        # Crear un nuevo registro por cada punto
        registros = []
        for punto in geom.geoms:
            nuevo = gdf_fila.iloc[0].copy()
            nuevo[gdf_fila.geometry.name] = punto
            registros.append(nuevo)
        return gpd.GeoDataFrame(registros, geometry=gdf_fila.geometry.name, crs=gdf_fila.crs)
    else:
        # Si ya es un punto, retornar tal cual
        return gdf_fila.copy()

def definir_lotes_solicitud(gdf_puntos_indi):
    id_os = gdf_puntos_indi.iloc[0]['id_os']
    codigo_ca = gdf_puntos_indi.iloc[0]['cod_ca']
    nombre_ca = gdf_puntos_indi.iloc[0]['nom_ca']
    semana_plan = gdf_puntos_indi.iloc[0]['semana_plan']
    
    # Crear un GeoDataFrame vacío para almacenar los puntos que no intersectan con ningún polígono de siembra
    puntos_indi_solicitado = gdf_puntos_indi[~gdf_puntos_indi.geometry.apply(lambda point: any(gdf_siembras.contains(point)))]
    
    # identifoca los lotes que se intersectan
    gdf_lotes_cat = gpd.sjoin(gdf_cat, puntos_indi_solicitado, how='inner', predicate='intersects')    
    
    # filtra los lotes intersectados en la capa original
    poligonos_intersect_cat = gdf_cat.loc[gdf_cat.index.isin(gdf_lotes_cat.index)]
    
    # copy
    poligonos_intersect_cat = poligonos_intersect_cat.copy()
    poligonos_intersect_cat = poligonos_intersect_cat[['unidad_01', 'unidad_02', 'unidad_03', 'unidad_04', 'unidad_05', 'area', 'soca', 'zona', 'geom']]
    poligonos_intersect_cat["soca"] = poligonos_intersect_cat["soca"].fillna(0)
    # Cambiar el nombre de la columna 'old_name' a 'new_name'
    poligonos_intersect_cat.rename(columns={'zona': 'inst'}, inplace=True)
    # CAMBIAR NOMBRE DATOS DE UNIDAD_03 Y UNDIAD_04 DE LOTES SOLICITADOS A LO INDICADO EN LOS PUNTOS
    poligonos_intersect_cat['unidad_03'] = codigo_ca
    poligonos_intersect_cat['unidad_04'] = nombre_ca
    poligonos_intersect_cat['semana_plan'] = semana_plan
    
    # agrega los campos faltantes
    poligonos_intersect_cat['origen'] = 'SOLICITUD'
    poligonos_intersect_cat['dias'] = 0
    poligonos_intersect_cat['fecha'] = ''
    poligonos_intersect_cat['os'] = id_os
    return poligonos_intersect_cat

def definir_lotes_siembra(gdf_puntos_indi):
    id_os = gdf_puntos_indi.iloc[0]['id_os']
    codigo_ca = gdf_puntos_indi.iloc[0]['cod_ca']
    nombre_ca = gdf_puntos_indi.iloc[0]['nom_ca']
    semana_plan = gdf_puntos_indi.iloc[0]['semana_plan']
    
    # SELECCIONAR LOS LOTES DE SIEMBRAS
    # identifoca los lotes que se intersectan
    gdf_lotes_siem = gpd.sjoin(gdf_siembras, gdf_puntos_indi, how='inner', predicate='intersects')
    # filtra los lotes intersectados en la capa original
    poligonos_intersect_siem = gdf_siembras.loc[gdf_siembras.index.isin(gdf_lotes_siem.index)]
    # copy
    poligonos_intersect_siem = poligonos_intersect_siem.copy()
    # agregar campo de semana_planificacion
    poligonos_intersect_siem['os'] = id_os
    # crea el campo soca, y coloca 0 o 1 dependiendo del origen
    poligonos_intersect_siem['soca'] = poligonos_intersect_siem['origen'].apply(lambda x: 1 if 'SIEMBRA 2024' in x else 0)

    # agrega los campos faltantes
    poligonos_intersect_siem['semana_plan'] = semana_plan
    
    # ordena los campos
    poligonos_intersect_siem = poligonos_intersect_siem[['unidad_01', 'unidad_02', 'unidad_03', 'unidad_04', 'unidad_05', 'area', 'soca', 'inst',
           'geom', 'origen', 'dias', 'fecha', 'os', 'semana_plan']]
    return poligonos_intersect_siem

def agregar_campos_faltantes(gdf_plan):
    gdf_plan['producto'] = ''
    gdf_plan['dosis'] = 0.0
    gdf_plan['id_labor'] = ''
    gdf_plan['fecha'] = ''
    gdf_plan['semana'] = 0
    gdf_plan['num_lib'] = 0
    gdf_plan['hora_ini'] = ''
    gdf_plan['hora_fin'] = ''
    gdf_plan['temp'] = 0
    gdf_plan['viento'] = 0
    gdf_plan['humedad'] = 0
    gdf_plan['cod_dron'] = ''
    gdf_plan['piloto_1'] = ''
    gdf_plan['piloto_2'] = ''
    gdf_plan['obs'] = ''
    gdf_plan['id_log'] = 0
    return gdf_plan

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['dias'] = gdf_pla_gral['dias'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_345601 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, etiqueta, producto, dosis, cantidad, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', '{str(round(row['area'],2)) + 'ha - ' + str(round(row['producto_cantidad'],2)) + 'ml'}', '{row['producto_plan']}', {row['producto_dosis']}, {row['producto_cantidad']}, ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
            """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        marcar_como_planificado(row['id'])
    return None

def cargar_a_amigocloud_semana_siguiente(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['dias'] = gdf_pla_gral['dias'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_381625 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, etiqueta, producto, dosis, cantidad, semana_plan, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', '{str(round(row['area'],2)) + 'ha - ' + str(round(row['producto_cantidad'],2)) + 'ml'}', '{row['producto_plan']}', {row['producto_dosis']}, {row['producto_cantidad']}, {row['semana_plan']}, ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
            """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f"Lotes cargado proxima semana ID: {row['id']}")
        # marcar_como_planificado(row['id'])
    return None

def cargar_a_amigocloud_lista_canhero_notif(gdf):
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf.iterrows():
        cod = row['unidad_03']
        nom = row['unidad_04']
        cod_canhero = str(cod) + ' / ' + nom
        insert_sql = f"insert into dataset_354656 (cod_canhero) values('{cod_canhero}')"
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f"Se ha registrado nuevo cañero para notificacion: {cod_canhero}")
        # marcar_como_planificado(row['id'])
    return None

# verifica si en la dataframe existen nombres de lotes repetidos
def validar_lotes_repetidos(gdf_plan):
    cods_props = list(set(gdf_plan['unidad_01']))
    for i in cods_props:
        prop = gdf_plan[gdf_plan['unidad_01'] == i]
        dup = prop[prop['unidad_05'].duplicated(keep=False)]
        if prop['unidad_05'].duplicated().any():
            print('Lotes repedidos:', list(set(dup['unidad_05'])))
            return True
        else:
            return False
    return None

def crear_kmls(gdf_lotes):
    lista_cods = list(set(gdf_lotes['unidad_01']))
    for i in lista_cods:
        prop = gdf_lotes[gdf_lotes['unidad_01'] == i]
        prop_wgs = prop.to_crs(epsg=4326)
        output_folder = PATH_KML + '\\' + str(i)
        print(output_folder)
        
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)  # Elimina toda la carpeta y su contenido
        os.makedirs(output_folder)  # La vuelve a crear vacía
        
        # Recorrer el GeoDataFrame
        for idx, row in prop_wgs.iterrows():
            # Crear un nuevo documento KML
            kml_doc = simplekml.Kml()
            # Obtener el polígono y el nombre
            polygon = row['geom']
            name = row['unidad_05']
            # Convertir el polígono en una lista de coordenadas
            if polygon.geom_type == "Polygon":
                coords = [(x, y) for x, y in zip(*polygon.exterior.xy)]
            elif polygon.geom_type == "MultiPolygon":
            # puedes iterar si quieres más de uno, aquí tomamos el primero
                first_poly = list(polygon.geoms)[0]
                coords = [(x, y) for x, y in zip(*first_poly.exterior.xy)]
            else:
                print(f"❌ Geometría no compatible: {polygon.geom_type}")
                continue  # salta este registro
            # Añadir el polígono al documento KML
            pol = kml_doc.newpolygon(name=name, outerboundaryis=coords)
            # Guardar el archivo KML
            file_name = f"{name}.kml"
            output_path = os.path.join(output_folder, file_name)
            kml_doc.save(output_path)
    return None

In [27]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

# ACTUALIZAR SHP PLANIFICACION

In [28]:
gdf_siembras = get_siembras()
gdf_cat = get_catastro()

In [29]:
gdf_puntos = obtener_planificacion_no_procesado()

In [30]:
gdf_puntos

,id_os,geom,cod_ca,nom_ca,obs,procesado,producto,dosis,semana_plan
0,203,"MULTIPOINT (471906.161 8089631.419, 472142.249...",794,AGUILERA WENDE MARIANO,None,False,trichogramma,6.0,42
1,219,"MULTIPOINT (473952.001 8093442.650, 474271.769...",794,AGUILERA WENDE MARIANO,None,False,trichogramma,6.0,42


In [31]:
for i, row in gdf_puntos.iterrows():
    ors = row['id_os']
    produc_plan = row['producto']
    produc_dosis = row['dosis']
    
    puntos = filtrar_os_y_quitar_caracteres(gdf_puntos, ors)
    multipoint_gdf = puntos.copy()
    puntos_individuales = crear_puntos_individuales(multipoint_gdf)
    
    lotes_solicidud = definir_lotes_solicitud(puntos_individuales)
    lotes_siembra = definir_lotes_siembra(puntos_individuales)
    
    # verificar si algun dataframe esta vacio
    dfs = [df for df in [lotes_solicidud, lotes_siembra] if not df.empty]
    # concatena los dfs no vacios
    concat_plan = pd.concat(dfs, axis=0, ignore_index=True)
    
    plan = agregar_campos_faltantes(concat_plan)
    #RECALCULAR EL AREA
    plan['area'] = plan.geom.area / 10000

    plan['producto_plan'] = produc_plan
    plan['producto_dosis'] = produc_dosis
    plan['producto_cantidad'] = plan['area'] * plan['producto_dosis']
    
    #plan.rename(columns={'geom': 'geom'}, inplace=True)
    #plan = plan.set_geometry("geom")
    plan['unidad_01'] = plan['unidad_01'].astype(int)
    plan['unidad_03'] = plan['unidad_03'].astype(int)
    plan['dias'] = plan['dias'].astype(int)
    plan['os'] = plan['os'].astype(int)
    plan['soca'] = plan['soca'].astype(int)
    plan['inst'] = plan['inst'].astype(int)

    # validas que la planificacion no tenga lotes con el mismo nombre
    if validar_lotes_repetidos(plan):
        print(f"ERROR: Existen lotes con nombre repetido en la propiedad: {plan.iloc[0]['unidad_01']}")
        continue
    #crear_kmls(plan)    
    plan.to_postgis("parte_diario_ctrl_bio", obtener_engine(), schema="drones_control_bio", if_exists="append")
    
    marcar_como_procesado(ors)

✔️ id_os 203 marcado como procesado.
✔️ id_os 219 marcado como procesado.


## Cargar a AmigoCloud los lotes sin planificar

In [32]:
lotes_sin_planificar = obtener_parte_diario_sin_planificar()
print('Lotes sin planificar:', len(lotes_sin_planificar))

Lotes sin planificar: 14


In [33]:
cargar_a_amigocloud(lotes_sin_planificar)

✔️ id_lote 2230 marcado como PLANIFICADO.
✔️ id_lote 2231 marcado como PLANIFICADO.
✔️ id_lote 2232 marcado como PLANIFICADO.
✔️ id_lote 2233 marcado como PLANIFICADO.
✔️ id_lote 2234 marcado como PLANIFICADO.
✔️ id_lote 2235 marcado como PLANIFICADO.
✔️ id_lote 2236 marcado como PLANIFICADO.
✔️ id_lote 2237 marcado como PLANIFICADO.
✔️ id_lote 2238 marcado como PLANIFICADO.
✔️ id_lote 2239 marcado como PLANIFICADO.
✔️ id_lote 2240 marcado como PLANIFICADO.
✔️ id_lote 2241 marcado como PLANIFICADO.
✔️ id_lote 2242 marcado como PLANIFICADO.
✔️ id_lote 2243 marcado como PLANIFICADO.


In [49]:
lotes_planificados = obtener_parte_diario_planificado()

In [50]:
lotes_planificados = obtener_parte_diario_planificado()
lotes_planificados = lotes_planificados[['unidad_03', 'unidad_04']]
lotes_planificados = lotes_planificados.drop_duplicates(subset=['unidad_03'])
lotes_planificados

,unidad_03,unidad_04
0,12162,ROCA REYES MARCOS
2,11860,ROCA SERRANO ARMANDO
9,18189,BALCAZAR SOLETO RAMON BERTHY
10,594,AGROPECUARIA FLAMBOYAN
15,13067,SAUCEDO CERRUTI SONIA
22,10697,PAZ REA JAIME EDUARDO
25,10863,PAZ REA ROSENDO
32,42231,PAZ FERNANDEZ RODRIGO
38,515,AGUILERA OLGA RIVERO VDA DE
41,8003,AGUILERA RIVERO SERGIO STEFAN


In [51]:
cargar_a_amigocloud_lista_canhero_notif(lotes_planificados)

Se ha registrado nuevo cañero para notificacion: 12162 / ROCA REYES MARCOS
Se ha registrado nuevo cañero para notificacion: 11860 / ROCA SERRANO ARMANDO
Se ha registrado nuevo cañero para notificacion: 18189 / BALCAZAR SOLETO RAMON BERTHY
Se ha registrado nuevo cañero para notificacion: 594 / AGROPECUARIA FLAMBOYAN
Se ha registrado nuevo cañero para notificacion: 13067 / SAUCEDO CERRUTI SONIA
Se ha registrado nuevo cañero para notificacion: 10697 / PAZ REA JAIME EDUARDO
Se ha registrado nuevo cañero para notificacion: 10863 / PAZ REA ROSENDO
Se ha registrado nuevo cañero para notificacion: 42231 / PAZ FERNANDEZ RODRIGO
Se ha registrado nuevo cañero para notificacion: 515 / AGUILERA OLGA RIVERO VDA DE
Se ha registrado nuevo cañero para notificacion: 8003 / AGUILERA RIVERO SERGIO STEFAN
Se ha registrado nuevo cañero para notificacion: 40004 / AGUILERA RIVERO NICOLAS JULIO
Se ha registrado nuevo cañero para notificacion: 794 / AGUILERA WENDE MARIANO
Se ha registrado nuevo cañero para noti

## Cargar a otro dataset los lotes sin planificar

In [9]:
cargar_a_amigocloud_semana_siguiente(lotes_sin_planificar)

Lotes cargado proxima semana ID: 1688
Lotes cargado proxima semana ID: 1689
Lotes cargado proxima semana ID: 1690
Lotes cargado proxima semana ID: 1691
Lotes cargado proxima semana ID: 1692
Lotes cargado proxima semana ID: 1693
Lotes cargado proxima semana ID: 1694
Lotes cargado proxima semana ID: 1695
Lotes cargado proxima semana ID: 1696
Lotes cargado proxima semana ID: 1697
Lotes cargado proxima semana ID: 1698
Lotes cargado proxima semana ID: 1699
Lotes cargado proxima semana ID: 1700
Lotes cargado proxima semana ID: 1701
Lotes cargado proxima semana ID: 1702
Lotes cargado proxima semana ID: 1703
Lotes cargado proxima semana ID: 1704
Lotes cargado proxima semana ID: 1705
Lotes cargado proxima semana ID: 1706
Lotes cargado proxima semana ID: 1707
Lotes cargado proxima semana ID: 1708
Lotes cargado proxima semana ID: 1709


## Crear KML de lotes planificados

In [52]:
lotes_planificados = obtener_parte_diario_planificado()

In [53]:
crear_kmls(lotes_planificados)

G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\256
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\1411
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\519
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\776
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\1290
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\10
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\270
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\16
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\17
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Tr